In [5]:
import pandas as pd

In [7]:
new_switch_df = pd.read_excel("./ver160用最終_4実8推プレ_工場月度割り当て_パラメータ田村さん記載.xlsx",sheet_name="切り替え時間（新）")
new_switch_df

,工場,品種1,品種2,切り替え時間,メモ（⇒、⇐、【補足説明】）
0,L1,2CT1W,2SANSWL,181.0,行きも帰りも同一時間
1,L1,2CT1W,4USQW,181.0,行きも帰りも同一時間
2,L1,2CT1W,SWP原反,200.0,行きも帰りも同一時間
3,L1,2SANSWL,4USQW,36.0,行きも帰りも同一時間
4,L1,2SANSWL,SWP原反,48.0,行きも帰りも同一時間
...,...,...,...,...,...
66,L6,3XR-1UWS,SAZMA原反(W_EXB),504.0,行きも帰りも同一時間
67,L6,SAZMA-S原反(W_EXB),SAZMA原反(W_EXB),36.0,行きも帰りも同一時間
68,L7,3PR-1EUW,3XR-1EUW,5.0,行きも帰りも同一時間
69,L7,3PR-1EUW,3XR-1UWD,5.5,行きと帰りの切替時間の平均値


In [10]:
l1_switch = new_switch_df[new_switch_df["工場"]=="L1"]
l1_switch

,工場,品種1,品種2,切り替え時間,メモ（⇒、⇐、【補足説明】）
0,L1,2CT1W,2SANSWL,181.0,行きも帰りも同一時間
1,L1,2CT1W,4USQW,181.0,行きも帰りも同一時間
2,L1,2CT1W,SWP原反,200.0,行きも帰りも同一時間
3,L1,2SANSWL,4USQW,36.0,行きも帰りも同一時間
4,L1,2SANSWL,SWP原反,48.0,行きも帰りも同一時間
5,L1,4USQW,SWP原反,48.0,行きも帰りも同一時間


In [12]:
import pandas as pd
import numpy as np

df = l1_switch

# ピボットテーブルを作成して、距離行列を表現
distance_matrix = df.pivot(index='品種1', columns='品種2', values='切り替え時間')

# 距離行列は対称行列であるべきなので、下三角行列を上三角行列の値で埋めます
distance_matrix = distance_matrix.combine_first(distance_matrix.T)

# 対角成分は同一品種間の距離なので、0で埋めます
np.fill_diagonal(distance_matrix.values, 0)

# 結果の距離行列を表示
print(distance_matrix)

         2CT1W  2SANSWL  4USQW  SWP原反
2CT1W      0.0    181.0  181.0  200.0
2SANSWL  181.0      0.0   36.0   48.0
4USQW    181.0     36.0    0.0   48.0
SWP原反    200.0     48.0   48.0    0.0


In [7]:
rare_df = pd.read_excel("./ver370用最終_4実8推プレ_工場月度割り当て_パラメータ田村さん記載.xlsx",sheet_name="レア品種")

In [8]:
rare_df

,工場,品種
0,L1,4USQW
1,L2,4UYW
2,L3,8UAW


In [9]:
row_tuples = list(rare_df.itertuples(index=False, name=None))
print(row_tuples)

[('L1', '4USQW'), ('L2', '4UYW'), ('L3', '8UAW')]


In [2]:
import pulp

# バイナリ変数vを定義
v = [pulp.LpVariable(f'v_{i}', cat='Binary') for i in range(6)]

# 距離行列Dの定義
D = [
    [0, 2, 3, 4, 1, 5],
    [2, 0, 1, 3, 2, 4],
    [3, 1, 0, 2, 3, 5],
    [4, 3, 2, 0, 4, 1],
    [1, 2, 3, 4, 0, 2],
    [5, 4, 5, 1, 2, 0]
]

# 新しい変数zの定義
z = [[pulp.LpVariable(f'z_{i}_{j}', cat='Binary') for j in range(6)] for i in range(6)]

# 問題の定義
problem = pulp.LpProblem("HadamardProductOptimization", pulp.LpMaximize)

# 目的関数にアダマール積の合計を設定
problem += pulp.lpSum(z[i][j] * D[i][j] for i in range(6) for j in range(6))

# 制約の設定
for i in range(6):
    for j in range(6):
        problem += z[i][j] <= v[i]
        problem += z[i][j] <= v[j]
        problem += z[i][j] >= v[i] + v[j] - 1

# 問題を解く
problem.solve()

# 結果の表示
print("Values of z:")
for i in range(6):
    print([pulp.value(z[i][j]) for j in range(6)])

print("Values of v:", [pulp.value(var) for var in v])


Values of z:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Values of v: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [3]:
import pulp

# 製品と段取り時間を定義
products = ['A', 'B', 'C']
setup_time = {
    ('A', 'B'): 2,
    ('A', 'C'): 3,
    ('B', 'A'): 2,
    ('B', 'C'): 1,
    ('C', 'A'): 3,
    ('C', 'B'): 2
}

# 最適化問題の定義
prob = pulp.LpProblem("Production_Schedule_Optimization_with_Selection", pulp.LpMinimize)

# 決定変数
y = pulp.LpVariable.dicts("y", ((i, t) for i in products for t in [1, 2]), 0, 1, pulp.LpBinary)  # 製品が月に選ばれるか
x = pulp.LpVariable.dicts("x", ((i, j, t) for i in products for j in products for t in [1, 2]), 0, 1, pulp.LpBinary)  # 製品間の順序

# 目的関数: 段取り時間の最小化
prob += pulp.lpSum([x[(i, j, t)] * setup_time.get((i, j), 0) for i in products for j in products for t in [1, 2] if i != j])

# 各月で選ばれる製品の数に制約を設定（例: 月1は3製品、月2は2製品）
prob += pulp.lpSum([y[(i, 1)] for i in products]) <= 3
prob += pulp.lpSum([y[(i, 2)] for i in products]) <= 2

# 各製品は月1か月2で1回だけ生産される制約
for i in products:
    prob += y[(i, 1)] + y[(i, 2)] <= 1

# 選ばれた製品同士の間でのみ順序を決定
for i in products:
    for j in products:
        for t in [1, 2]:
            if i != j:
                prob += x[(i, j, t)] <= y[(i, t)]
                prob += x[(i, j, t)] <= y[(j, t)]

# 問題を解く
prob.solve()

# 結果の表示
selected_products_month1 = [i for i in products if pulp.value(y[(i, 1)]) == 1]
selected_products_month2 = [i for i in products if pulp.value(y[(i, 2)]) == 1]

optimal_order = [(i, j, t) for i in products for j in products for t in [1, 2] if pulp.value(x[(i, j, t)]) == 1]
optimal_value = pulp.value(prob.objective)

print("Selected Products for Month 1: ", selected_products_month1)
print("Selected Products for Month 2: ", selected_products_month2)
print("Optimal Order: ", optimal_order)
print("Optimal Total Setup Time: ", optimal_value)

Selected Products for Month 1:  []
Selected Products for Month 2:  []
Optimal Order:  []
Optimal Total Setup Time:  0.0


In [1]:
import pulp

# バイナリ変数vを定義
v = [pulp.LpVariable(f'v_{i}', cat='Binary') for i in range(6)]

# 距離行列Dの定義
D = [
    [0, 2, 3, 4, 1, 5],
    [2, 0, 1, 3, 2, 4],
    [3, 1, 0, 2, 3, 5],
    [4, 3, 2, 0, 4, 1],
    [1, 2, 3, 4, 0, 2],
    [5, 4, 5, 1, 2, 0]
]

# 新しい変数zの定義
z = [[pulp.LpVariable(f'z_{i}_{j}', cat='Binary') for j in range(6)] for i in range(6)]

# 問題の定義
problem = pulp.LpProblem("HadamardProductOptimization", pulp.LpMaximize)

# 目的関数にアダマール積の合計を設定
problem += pulp.lpSum(z[i][j] * D[i][j] for i in range(6) for j in range(6))

# 制約の設定
for i in range(6):
    for j in range(6):
        problem += z[i][j] <= v[i]
        problem += z[i][j] <= v[j]
        problem += z[i][j] >= v[i] + v[j] - 1

# 問題を解く
problem.solve()

# 結果の表示
print("Values of z:")
for i in range(6):
    print([pulp.value(z[i][j]) for j in range(6)])

print("Values of v:", [pulp.value(var) for var in v])


Values of z:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Values of v: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [3]:
import pulp

# バイナリ変数vを定義
v = [pulp.LpVariable(f'v_{i}', cat='Binary') for i in range(6)]

# 距離行列Dの定義
D = [
    [0, 2, 3, 4, 1, 5],
    [2, 0, 1, 3, 2, 4],
    [3, 1, 0, 2, 3, 5],
    [4, 3, 2, 0, 4, 1],
    [1, 2, 3, 4, 0, 2],
    [5, 4, 5, 1, 2, 0]
]

# 新しい変数zの定義
z = [[pulp.LpVariable(f'z_{i}_{j}', cat='Binary') for j in range(6)] for i in range(6)]

# 目的関数を最大化するための新しい変数R
R = pulp.LpVariable("R")

# 問題の定義
problem = pulp.LpProblem("HadamardProductOptimization", pulp.LpMaximize)

# アダマール積の合計を計算
S = pulp.lpSum(z[i][j] * D[i][j] for i in range(6) for j in range(6))

# 目的関数を設定（Rを最大化）
problem += R

# 制約の設定
for i in range(6):
    for j in range(6):
        problem += z[i][j] <= v[i]
        problem += z[i][j] <= v[j]
        problem += z[i][j] >= v[i] + v[j] - 1

# 新しい制約を追加
T = pulp.lpSum(v)  # vの合計
problem += R * T <= S  # R * T ≤ S
problem += T >= 1  # vの合計が1以上

# Rを最大化するための制約を設定
problem += R >= 0  # Rは0以上

# 問題を解く
problem.solve()

# 結果の表示
print("Values of z:")
for i in range(6):
    print([pulp.value(z[i][j]) for j in range(6)])

print("Values of v:", [pulp.value(var) for var in v])
print("Optimal value of R:", pulp.value(R))



TypeError: Non-constant expressions cannot be multiplied

In [4]:
import pulp

# バイナリ変数vを定義
v = [pulp.LpVariable(f'v_{i}', cat='Binary') for i in range(6)]

# 距離行列Dの定義
D = [
    [0, 2, 3, 4, 1, 5],
    [2, 0, 1, 3, 2, 4],
    [3, 1, 0, 2, 3, 5],
    [4, 3, 2, 0, 4, 1],
    [1, 2, 3, 4, 0, 2],
    [5, 4, 5, 1, 2, 0]
]

# 新しい変数zの定義
z = [[pulp.LpVariable(f'z_{i}_{j}', cat='Binary') for j in range(6)] for i in range(6)]

# アダマール積の合計を計算
S = pulp.lpSum(z[i][j] * D[i][j] for i in range(6) for j in range(6))

# 問題の定義
problem = pulp.LpProblem("HadamardProductOptimization", pulp.LpMaximize)

# 目的関数にアダマール積の合計を設定
problem += S  # Sを最大化する

# 制約の設定
for i in range(6):
    for j in range(6):
        problem += z[i][j] <= v[i]
        problem += z[i][j] <= v[j]
        problem += z[i][j] >= v[i] + v[j] - 1

# バイナリ変数の合計を制約として設定
T = pulp.lpSum(v)  # vの合計

# SをTで割ったものを新しい変数Rに等しく設定
R = pulp.LpVariable("R")

# S ≤ R * Tという制約を追加
problem += S <= R * T
problem += T >= 1  # Tが1以上であること

# 問題を解く
problem.solve()

# 結果の表示
print("Values of z:")
for i in range(6):
    print([pulp.value(z[i][j]) for j in range(6)])

print("Values of v:", [pulp.value(var) for var in v])
print("Optimal value of R:", pulp.value(R))


TypeError: Non-constant expressions cannot be multiplied

In [5]:
switching_combination ={
    (): 0,
    ('A',): 0,
    ('B',): 0,
    ('C',): 0,
    ('D',): 0,
    ('E',): 0,
    ('A', 'B'): 36,
    ('A', 'C'): 91,
    ('A', 'D'): 70,
    ('A', 'E'): 45,
    ('B', 'C'): 80,
    ('B', 'D'): 4,
    ('B', 'E'): 9,
    ('C', 'D'): 22,
    ('C', 'E'): 55,
    ('D', 'E'): 18,
    ('A', 'B', 'C'): 63,
    ('A', 'B', 'D'): 15,
    ('A', 'B', 'E'): 38,
    ('A', 'C', 'D'): 71,
    ('A', 'C', 'E'): 34,
    ('A', 'D', 'E'): 13,
    ('B', 'C', 'D'): 67,
    ('B', 'C', 'E'): 24,
    ('B', 'D', 'E'): 56,
    ('C', 'D', 'E'): 40,
    ('A', 'B', 'C', 'D'): 99,
    ('A', 'B', 'C', 'E'): 46,
    ('A', 'B', 'D', 'E'): 79,
    ('A', 'C', 'D', 'E'): 23,
    ('B', 'C', 'D', 'E'): 11,
    ('A', 'B', 'C', 'D', 'E'): 82
}


In [12]:
import pulp

# 1. switching_combination の辞書を使用
switching_combination = {
    frozenset(()): 0,
    frozenset({'A'}): 0,
    frozenset({'B'}): 0,
    frozenset({'C'}): 0,
    frozenset({'D'}): 0,
    frozenset({'E'}): 0,
    frozenset({'A', 'B'}): 36,
    frozenset({'A', 'C'}): 91,
    frozenset({'A', 'D'}): 70,
    frozenset({'A', 'E'}): 45,
    frozenset({'B', 'C'}): 80,
    frozenset({'B', 'D'}): 4,
    frozenset({'B', 'E'}): 9,
    frozenset({'C', 'D'}): 22,
    frozenset({'C', 'E'}): 55,
    frozenset({'D', 'E'}): 18,
    frozenset({'A', 'B', 'C'}): 63,
    frozenset({'A', 'B', 'D'}): 15,
    frozenset({'A', 'B', 'E'}): 38,
    frozenset({'A', 'C', 'D'}): 71,
    frozenset({'A', 'C', 'E'}): 34,
    frozenset({'A', 'D', 'E'}): 13,
    frozenset({'B', 'C', 'D'}): 67,
    frozenset({'B', 'C', 'E'}): 24,
    frozenset({'B', 'D', 'E'}): 56,
    frozenset({'C', 'D', 'E'}): 40,
    frozenset({'A', 'B', 'C', 'D'}): 99,
    frozenset({'A', 'B', 'C', 'E'}): 46,
    frozenset({'A', 'B', 'D', 'E'}): 79,
    frozenset({'A', 'C', 'D', 'E'}): 23,
    frozenset({'B', 'C', 'D', 'E'}): 11,
    frozenset({'A', 'B', 'C', 'D', 'E'}): 82
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのバイナリ変数を辞書で定義
var_names = ['A', 'B', 'C', 'D', 'E']
x = pulp.LpVariable.dicts("x", var_names, cat='Binary')

# 3. 目的関数の定義
# バイナリ変数の組み合わせに対応する switching_combination の値を目的関数として設定
objective = pulp.lpSum(
    switching_combination.get(frozenset({name for name in var_names if pulp.value(x[name]) == 1}), 0)
    for subset in switching_combination.keys()
)

prob += objective

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum([x[name] for name in var_names]) >= 3

# 5. 問題を解く
prob.solve()

# 6. 結果の出力
print("Optimal production decision:")
for name in var_names:
    print(f"{name}: {pulp.value(x[name])}")

print(f"Switching Combination Value: {pulp.value(objective)}")


Optimal production decision:
A: 1.0
B: 1.0
C: 1.0
D: 0.0
E: 0.0
Switching Combination Value: None


In [13]:
import pulp

# 1. switching_combination の辞書を使用
switching_combination = {
    frozenset(()): 0,
    frozenset({'A'}): 0,
    frozenset({'B'}): 0,
    frozenset({'C'}): 0,
    frozenset({'D'}): 0,
    frozenset({'E'}): 0,
    frozenset({'A', 'B'}): 36,
    frozenset({'A', 'C'}): 91,
    frozenset({'A', 'D'}): 70,
    frozenset({'A', 'E'}): 45,
    frozenset({'B', 'C'}): 80,
    frozenset({'B', 'D'}): 4,
    frozenset({'B', 'E'}): 9,
    frozenset({'C', 'D'}): 22,
    frozenset({'C', 'E'}): 55,
    frozenset({'D', 'E'}): 18,
    frozenset({'A', 'B', 'C'}): 63,
    frozenset({'A', 'B', 'D'}): 15,
    frozenset({'A', 'B', 'E'}): 38,
    frozenset({'A', 'C', 'D'}): 71,
    frozenset({'A', 'C', 'E'}): 34,
    frozenset({'A', 'D', 'E'}): 13,
    frozenset({'B', 'C', 'D'}): 67,
    frozenset({'B', 'C', 'E'}): 24,
    frozenset({'B', 'D', 'E'}): 56,
    frozenset({'C', 'D', 'E'}): 40,
    frozenset({'A', 'B', 'C', 'D'}): 99,
    frozenset({'A', 'B', 'C', 'E'}): 46,
    frozenset({'A', 'B', 'D', 'E'}): 79,
    frozenset({'A', 'C', 'D', 'E'}): 23,
    frozenset({'B', 'C', 'D', 'E'}): 11,
    frozenset({'A', 'B', 'C', 'D', 'E'}): 82
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのバイナリ変数を辞書で定義
var_names = ['A', 'B', 'C', 'D', 'E']
x = pulp.LpVariable.dicts("x", var_names, cat='Binary')

# 3. 目的関数の定義
# バイナリ変数の組み合わせに対応する switching_combination の値を目的関数として設定
objective = pulp.lpSum(
    value * pulp.lpProduct([x[name] for name in subset]) 
    for subset, value in switching_combination.items()
)

prob += objective

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum([x[name] for name in var_names]) >= 1

# 5. 問題を解く
prob.solve()

# 6. 結果の出力
print("Optimal production decision:")
for name in var_names:
    print(f"{name}: {pulp.value(x[name])}")

print(f"Switching Combination Value: {pulp.value(prob.objective)}")


AttributeError: module 'pulp' has no attribute 'lpProduct'

In [14]:
import pulp

# 1. switching_combination の辞書を使用
switching_combination = {
    frozenset(()): 0,
    frozenset({'A'}): 0,
    frozenset({'B'}): 0,
    frozenset({'C'}): 0,
    frozenset({'D'}): 0,
    frozenset({'E'}): 0,
    frozenset({'A', 'B'}): 36,
    frozenset({'A', 'C'}): 91,
    frozenset({'A', 'D'}): 70,
    frozenset({'A', 'E'}): 45,
    frozenset({'B', 'C'}): 80,
    frozenset({'B', 'D'}): 4,
    frozenset({'B', 'E'}): 9,
    frozenset({'C', 'D'}): 22,
    frozenset({'C', 'E'}): 55,
    frozenset({'D', 'E'}): 18,
    frozenset({'A', 'B', 'C'}): 63,
    frozenset({'A', 'B', 'D'}): 15,
    frozenset({'A', 'B', 'E'}): 38,
    frozenset({'A', 'C', 'D'}): 71,
    frozenset({'A', 'C', 'E'}): 34,
    frozenset({'A', 'D', 'E'}): 13,
    frozenset({'B', 'C', 'D'}): 67,
    frozenset({'B', 'C', 'E'}): 24,
    frozenset({'B', 'D', 'E'}): 56,
    frozenset({'C', 'D', 'E'}): 40,
    frozenset({'A', 'B', 'C', 'D'}): 99,
    frozenset({'A', 'B', 'C', 'E'}): 46,
    frozenset({'A', 'B', 'D', 'E'}): 79,
    frozenset({'A', 'C', 'D', 'E'}): 23,
    frozenset({'B', 'C', 'D', 'E'}): 11,
    frozenset({'A', 'B', 'C', 'D', 'E'}): 82
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのバイナリ変数を辞書で定義
var_names = ['A', 'B', 'C', 'D', 'E']
x = pulp.LpVariable.dicts("x", var_names, cat='Binary')

# 3. 目的関数の定義
# バイナリ変数の積を表現するために、各 subset のすべての要素が 1 になるような組み合わせで値を加える
objective = pulp.lpSum(
    value * pulp.lpProduct([x[name] for name in subset]) 
    for subset, value in switching_combination.items()
    if subset  # 空の部分集合は無視
)

prob += objective

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum([x[name] for name in var_names]) >= 1

# 5. 問題を解く
prob.solve()

# 6. 結果の出力
print("Optimal production decision:")
for name in var_names:
    print(f"{name}: {pulp.value(x[name])}")

print(f"Switching Combination Value: {pulp.value(prob.objective)}")


AttributeError: module 'pulp' has no attribute 'lpProduct'

In [15]:
import pulp
import numpy as np

# 1. switching_combination の辞書を使用
switching_combination = {
    frozenset(()): 0,
    frozenset({'A'}): 0,
    frozenset({'B'}): 0,
    frozenset({'C'}): 0,
    frozenset({'D'}): 0,
    frozenset({'E'}): 0,
    frozenset({'A', 'B'}): 36,
    frozenset({'A', 'C'}): 91,
    frozenset({'A', 'D'}): 70,
    frozenset({'A', 'E'}): 45,
    frozenset({'B', 'C'}): 80,
    frozenset({'B', 'D'}): 4,
    frozenset({'B', 'E'}): 9,
    frozenset({'C', 'D'}): 22,
    frozenset({'C', 'E'}): 55,
    frozenset({'D', 'E'}): 18,
    frozenset({'A', 'B', 'C'}): 63,
    frozenset({'A', 'B', 'D'}): 15,
    frozenset({'A', 'B', 'E'}): 38,
    frozenset({'A', 'C', 'D'}): 71,
    frozenset({'A', 'C', 'E'}): 34,
    frozenset({'A', 'D', 'E'}): 13,
    frozenset({'B', 'C', 'D'}): 67,
    frozenset({'B', 'C', 'E'}): 24,
    frozenset({'B', 'D', 'E'}): 56,
    frozenset({'C', 'D', 'E'}): 40,
    frozenset({'A', 'B', 'C', 'D'}): 99,
    frozenset({'A', 'B', 'C', 'E'}): 46,
    frozenset({'A', 'B', 'D', 'E'}): 79,
    frozenset({'A', 'C', 'D', 'E'}): 23,
    frozenset({'B', 'C', 'D', 'E'}): 11,
    frozenset({'A', 'B', 'C', 'D', 'E'}): 82
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのバイナリ変数を辞書で定義
var_names = ['A', 'B', 'C', 'D', 'E']
x = pulp.LpVariable.dicts("x", var_names, cat='Binary')

# 3. 目的関数の定義
# 手動で積を計算するために numpy.prod を使用
objective = pulp.lpSum(
    value * np.prod([x[name] for name in subset])
    for subset, value in switching_combination.items()
    if subset  # 空集合は無視
)

prob += objective

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum([x[name] for name in var_names]) >= 1

# 5. 問題を解く
prob.solve()

# 6. 結果の出力
print("Optimal production decision:")
for name in var_names:
    print(f"{name}: {pulp.value(x[name])}")

print(f"Switching Combination Value: {pulp.value(prob.objective)}")


TypeError: Non-constant expressions cannot be multiplied

In [17]:
import pulp

# 1. switching_combination の辞書を使用
# リスト形式で各部分集合を定義（品種A～Eに対応するバイナリ表現）
switching_combination = {
    (0, 0, 0, 0, 0): 0,
    (1, 0, 0, 0, 0): 0,
    (0, 1, 0, 0, 0): 0,
    (0, 0, 1, 0, 0): 0,
    (0, 0, 0, 1, 0): 0,
    (0, 0, 0, 0, 1): 0,
    (1, 1, 0, 0, 0): 36,
    (1, 0, 1, 0, 0): 91,
    (1, 0, 0, 1, 0): 70,
    (1, 0, 0, 0, 1): 45,
    (0, 1, 1, 0, 0): 80,
    (0, 1, 0, 1, 0): 4,
    (0, 1, 0, 0, 1): 9,
    (0, 0, 1, 1, 0): 22,
    (0, 0, 1, 0, 1): 55,
    (0, 0, 0, 1, 1): 18,
    (1, 1, 1, 0, 0): 63,
    (1, 1, 0, 1, 0): 15,
    (1, 1, 0, 0, 1): 38,
    (1, 0, 1, 1, 0): 71,
    (1, 0, 1, 0, 1): 34,
    (1, 0, 0, 1, 1): 13,
    (0, 1, 1, 1, 0): 67,
    (0, 1, 1, 0, 1): 24,
    (0, 1, 0, 1, 1): 56,
    (0, 0, 1, 1, 1): 40,
    (1, 1, 1, 1, 0): 99,
    (1, 1, 1, 0, 1): 46,
    (1, 1, 0, 1, 1): 79,
    (1, 0, 1, 1, 1): 23,
    (0, 1, 1, 1, 1): 11,
    (1, 1, 1, 1, 1): 82
}

# 2. バイナリ変数の定義（リスト化されたバイナリ変数）
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのインデックス対応
var_names = ['A', 'B', 'C', 'D', 'E']
x = [pulp.LpVariable(f"x_{i}", cat='Binary') for i in range(len(var_names))]

# 3. 目的関数の定義
# リストとしてのバイナリ変数の値を取得し、対応する switching_combination の値を目的関数に設定
def get_combination_value():
    current_combination = tuple(int(pulp.value(var)) for var in x)
    return switching_combination.get(current_combination, 0)

# 4. 目的関数を最大化
prob += get_combination_value()

# 5. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum(x) >= 1

# 6. 問題を解く
prob.solve()

# 7. 結果の出力
print("Optimal production decision:")
for i, name in enumerate(var_names):
    print(f"{name}: {pulp.value(x[i])}")

# 8. 最適な switching_combination の値の出力
print(f"Switching Combination Value: {get_combination_value()}")


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [21]:
import pulp

# 1. switching_combination の辞書を使用
# リスト形式で各部分集合を定義（品種A～Eに対応するバイナリ表現）
switching_combination = {
    (0, 0, 0, 0, 0): 0,
    (1, 0, 0, 0, 0): 0,
    (0, 1, 0, 0, 0): 0,
    (0, 0, 1, 0, 0): 0,
    (0, 0, 0, 1, 0): 0,
    (0, 0, 0, 0, 1): 0,
    (1, 1, 0, 0, 0): 36,
    (1, 0, 1, 0, 0): 91,
    (1, 0, 0, 1, 0): 70,
    (1, 0, 0, 0, 1): 45,
    (0, 1, 1, 0, 0): 80,
    (0, 1, 0, 1, 0): 4,
    (0, 1, 0, 0, 1): 9,
    (0, 0, 1, 1, 0): 22,
    (0, 0, 1, 0, 1): 55,
    (0, 0, 0, 1, 1): 18,
    (1, 1, 1, 0, 0): 63,
    (1, 1, 0, 1, 0): 15,
    (1, 1, 0, 0, 1): 38,
    (1, 0, 1, 1, 0): 71,
    (1, 0, 1, 0, 1): 34,
    (1, 0, 0, 1, 1): 13,
    (0, 1, 1, 1, 0): 67,
    (0, 1, 1, 0, 1): 24,
    (0, 1, 0, 1, 1): 56,
    (0, 0, 1, 1, 1): 40,
    (1, 1, 1, 1, 0): 99,
    (1, 1, 1, 0, 1): 46,
    (1, 1, 0, 1, 1): 79,
    (1, 0, 1, 1, 1): 23,
    (0, 1, 1, 1, 1): 11,
    (1, 1, 1, 1, 1): 82
}

# 2. バイナリ変数の定義（リスト化されたバイナリ変数）
prob = pulp.LpProblem("Production_Problem", pulp.LpMinimize)

# 品種A～Eのインデックス対応
var_names = ['A', 'B', 'C', 'D', 'E']
x = [pulp.LpVariable(f"x_{i}", cat='Binary') for i in range(len(var_names))]

# 3. 目的関数の定義
# switching_combination の値に基づいた目的関数を線形和として表現
prob += - pulp.lpSum(
    switching_combination[combination] * pulp.lpSum(
        (x[i] if combination[i] == 1 else (1 - x[i])) for i in range(len(var_names))
    ) == len(var_names)  # 各品種が組み合わせ通りかをチェック
    for combination in switching_combination.keys()
)

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum(x) >= 1

# 5. 問題を解く
prob.solve()

# 6. 最適な組み合わせを取得
def get_combination_value():
    # 最適なバイナリ変数のリストを取得
    current_combination = tuple(int(pulp.value(var)) for var in x)
    return switching_combination.get(current_combination, 0)

# 7. 結果の出力
print("Optimal production decision:")
for i, name in enumerate(var_names):
    print(f"{name}: {pulp.value(x[i])}")

# 8. 最適な switching_combination の値の出力
print(f"Switching Combination Value: {get_combination_value()}")


Optimal production decision:
A: 1.0
B: 1.0
C: 1.0
D: 1.0
E: 0.0
Switching Combination Value: 99


In [22]:
import pulp

# 1. switching_combination の辞書を使用（バイナリ表現）
switching_combination = {
    (0, 0, 0, 0, 0): 0,
    (1, 0, 0, 0, 0): 0,
    (0, 1, 0, 0, 0): 0,
    (0, 0, 1, 0, 0): 0,
    (0, 0, 0, 1, 0): 0,
    (0, 0, 0, 0, 1): 0,
    (1, 1, 0, 0, 0): 36,
    (1, 0, 1, 0, 0): 91,
    (1, 0, 0, 1, 0): 70,
    (1, 0, 0, 0, 1): 45,
    (0, 1, 1, 0, 0): 80,
    (0, 1, 0, 1, 0): 4,
    (0, 1, 0, 0, 1): 9,
    (0, 0, 1, 1, 0): 22,
    (0, 0, 1, 0, 1): 55,
    (0, 0, 0, 1, 1): 18,
    (1, 1, 1, 0, 0): 63,
    (1, 1, 0, 1, 0): 15,
    (1, 1, 0, 0, 1): 38,
    (1, 0, 1, 1, 0): 71,
    (1, 0, 1, 0, 1): 34,
    (1, 0, 0, 1, 1): 13,
    (0, 1, 1, 1, 0): 67,
    (0, 1, 1, 0, 1): 24,
    (0, 1, 0, 1, 1): 56,
    (0, 0, 1, 1, 1): 40,
    (1, 1, 1, 1, 0): 99,
    (1, 1, 1, 0, 1): 46,
    (1, 1, 0, 1, 1): 79,
    (1, 0, 1, 1, 1): 23,
    (0, 1, 1, 1, 1): 11,
    (1, 1, 1, 1, 1): 82
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMaximize)

# 品種A～Eのインデックス対応
var_names = ['A', 'B', 'C', 'D', 'E']
x = [pulp.LpVariable(f"x_{i}", cat='Binary') for i in range(len(var_names))]

# 3. 目的関数の定義
# リスト化されたバイナリ変数の組み合わせを確認し、該当する switching_combination の値を目的関数として設定
def get_combination_value():
    current_combination = tuple(int(pulp.value(var)) for var in x)
    return switching_combination.get(current_combination, 0)

# 4. 目的関数の定義（最大化する目的関数）
prob += pulp.lpSum(
    value * (pulp.lpSum(
        [1 if int(pulp.value(x[i])) == combination[i] else 0 for i in range(len(var_names))]
    ) == len(var_names)) for combination, value in switching_combination.items()
)

# 5. 制約条件を追加 (少なくとも1つは生産される)
prob += pulp.lpSum(x) >= 1

# 6. 問題を解く
prob.solve()

# 7. 最適な組み合わせを取得
optimal_combination = get_combination_value()

# 8. 結果の出力
print("Optimal production decision:")
for i, name in enumerate(var_names):
    print(f"{name}: {pulp.value(x[i])}")

# 9. 最適な switching_combination の値の出力
print(f"Switching Combination Value: {optimal_combination}")


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [24]:
import pulp

# 1. 定数（switching_combinationのようなもの）
# リスト形式で各部分集合に対応する定数を定義
switching_combination = {
    (0, 0, 0, 0, 0): 0,
    (1, 0, 0, 0, 0): 5,  # 例: Aを生産するときの値
    (0, 1, 0, 0, 0): 10, # 例: Bを生産するときの値
    (1, 1, 0, 0, 0): 20, # 例: AとBを生産するときの値
    (0, 0, 1, 0, 0): 15, # 例: Cを生産するときの値
    (1, 0, 1, 0, 0): 25, # 例: AとCを生産するときの値
    (1, 1, 1, 0, 0): 30, # 例: A, B, Cを生産するときの値
    # 必要に応じて他の組み合わせも定義
}

# 2. バイナリ変数の定義
prob = pulp.LpProblem("Production_Problem", pulp.LpMinimize)

# 品種A～Eのインデックス対応
var_names = ['A', 'B', 'C', 'D', 'E']
x = [pulp.LpVariable(f"x_{i}", cat='Binary') for i in range(len(var_names))]

# 3. 目的関数の定義
# バイナリ変数と定数を掛け算して所望の値を取り出す方法
prob += -pulp.lpSum(
    value * pulp.lpSum(x[i] if combination[i] == 1 else (1 - x[i]) for i in range(len(var_names))) == len(var_names)
    for combination, value in switching_combination.items()
)

# 4. 制約条件を追加 (少なくとも1つは生産する)
prob += pulp.lpSum(x) >= 1

# 5. 問題を解く
prob.solve()

# 6. 結果の出力
print("Optimal production decision:")
for i, name in enumerate(var_names):
    print(f"{name}: {pulp.value(x[i])}")

# 最適な組み合わせの値を出力
optimal_combination = tuple(int(pulp.value(var)) for var in x)
print(f"Switching Combination Value: {switching_combination.get(optimal_combination, 0)}")


Optimal production decision:
A: 1.0
B: 1.0
C: 1.0
D: 0.0
E: 0.0
Switching Combination Value: 30


In [26]:
import itertools
import pulp

# 1. 単品種の生産を表現するバイナリ変数
var_names = ['A', 'B', 'C', 'D', 'E']
x = {name: pulp.LpVariable(f"x_{name}", cat='Binary') for name in var_names}

# 2. 2品種を同時に生産するかどうかを表現するバイナリ変数
# itertools.combinations を使用して2つの品種を選ぶ
x_pair = {(a, b): pulp.LpVariable(f"x_{a}_{b}", cat='Binary') for a, b in itertools.combinations(var_names, 2)}


In [27]:
import itertools
import pulp

# 1. 品種A～Eのインデックス対応
var_names = ['A', 'B', 'C', 'D', 'E']

# 2. バイナリ変数の定義
# 各品種ペアに対してバイナリ変数を定義
x_pair = {(a, b): pulp.LpVariable(f"x_{a}_{b}", cat='Binary') for a, b in itertools.combinations(var_names, 2)}

# 3. 線形問題の定義（最大化）
prob = pulp.LpProblem("Maximize_Production_Problem", pulp.LpMaximize)

# 4. 目的関数の定義（x_pair[a,b] と switching_combination[(a,b)] の積の総和）
# 積の総和を最大化する
objective = pulp.lpSum(x_pair[a, b] * switching_combination[frozenset({a, b})] for a, b in x_pair)
prob += objective

# 5. 制約条件（必要に応じて追加）
# 少なくとも1つのペアを選択する制約
prob += pulp.lpSum(x_pair[a, b] for a, b in x_pair) >= 1

# 6. 問題を解く
prob.solve()

# 7. 結果の出力
print("Optimal production decision for pairs:")
for (a, b) in x_pair:
    print(f"{a}, {b}: {pulp.value(x_pair[a, b])}")

# 8. 最適な目的関数の値（最大化された総和）を出力
print(f"Optimal objective value (total sum): {pulp.value(prob.objective)}")


KeyError: frozenset({'B', 'A'})

In [3]:
import pandas as pd
df = pd.read_excel("./Book1.xlsx")


init_switch_dict_set = {}

# Iterating through the DataFrame rows
for index, row in df.iterrows():
    factory = row['工場']
    task_pair = frozenset([row['タスク1'], row['タスク2']])  # Using frozenset to represent unordered pairs
    switch_time = row['切り替え時間']
    
    # If the factory is not in the dictionary, add it
    if factory not in init_switch_dict_set:
        init_switch_dict_set[factory] = {}
    
    # Add the task pair and its switch time
    init_switch_dict_set[factory][task_pair] = switch_time

# Now the dictionary uses sets instead of tuples for task pairs
init_switch_dict_set

{'plant1': {frozenset({'A', 'B'}): 181,
  frozenset({'A', 'C'}): 181,
  frozenset({'A', 'D'}): 200,
  frozenset({'B', 'C'}): 36,
  frozenset({'B', 'D'}): 48,
  frozenset({'C', 'D'}): 48},
 'plant2': {frozenset({'A', 'B'}): 50,
  frozenset({'A', 'C'}): 44,
  frozenset({'A', 'D'}): 44,
  frozenset({'A', 'E'}): 44,
  frozenset({'B', 'C'}): 44,
  frozenset({'B', 'D'}): 37,
  frozenset({'B', 'E'}): 41,
  frozenset({'C', 'D'}): 38,
  frozenset({'C', 'E'}): 38,
  frozenset({'D', 'E'}): 24}}

In [6]:
import pandas as pd
from itertools import chain, combinations


def powerset(iterable):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))



# Modify the calculation to use frozensets instead of tuples for handling sets
def get_switch_time_set(factory_dict, pair):
    sorted_pair = frozenset(pair)
    return factory_dict[sorted_pair]

# Adjust the switch time calculation to handle unordered pairs with frozensets
def calculate_switch_time_set(factory_dict, subset):
    pairs = list(combinations(subset, 2))
    total_time = sum(get_switch_time_set(factory_dict, pair) for pair in pairs)
    average_time = total_time / len(pairs)
    return average_time * (len(subset) - 1)

# Now, we will retry creating the final dictionary with the updated helper function using sets
final_switch_dict_set = {}

# Iterate over each factory
for factory, tasks in init_switch_dict_set.items():
    # Get the unique set of tasks for the factory
    unique_tasks = set(task for pair in tasks for task in pair)
    
    # Generate the power set of the unique tasks
    task_powerset = powerset(unique_tasks)
    
    # Initialize the factory in the new dictionary
    final_switch_dict_set[factory] = {}
    
    # Iterate over each subset in the power set
    for subset in task_powerset:
        subset_frozenset = frozenset(subset)
        if len(subset) == 1:
            # Case ①: 1品種の場合は0
            final_switch_dict_set[factory][subset_frozenset] = 0
        elif len(subset) == 2:
            # Case ②: 2品種の場合はもとの辞書の該当の値
            final_switch_dict_set[factory][subset_frozenset] = get_switch_time_set(init_switch_dict_set[factory], subset)
        else:
            # Case ③: 3品種以上の場合
            final_switch_dict_set[factory][subset_frozenset] = calculate_switch_time_set(init_switch_dict_set[factory], subset)

# Display the result
#tools.display_dataframe_to_user(name="Final Switch Dictionary with Sets", dataframe=pd.DataFrame(final_switch_dict_set))

# Return the dictionary in raw form as well
final_switch_dict_set


{'plant1': {frozenset({'D'}): 0,
  frozenset({'C'}): 0,
  frozenset({'B'}): 0,
  frozenset({'A'}): 0,
  frozenset({'C', 'D'}): 48,
  frozenset({'B', 'D'}): 48,
  frozenset({'A', 'D'}): 200,
  frozenset({'B', 'C'}): 36,
  frozenset({'A', 'C'}): 181,
  frozenset({'A', 'B'}): 181,
  frozenset({'B', 'C', 'D'}): 88.0,
  frozenset({'A', 'C', 'D'}): 286.0,
  frozenset({'A', 'B', 'D'}): 286.0,
  frozenset({'A', 'B', 'C'}): 265.3333333333333,
  frozenset({'A', 'B', 'C', 'D'}): 347.0},
 'plant2': {frozenset({'D'}): 0,
  frozenset({'C'}): 0,
  frozenset({'B'}): 0,
  frozenset({'A'}): 0,
  frozenset({'E'}): 0,
  frozenset({'C', 'D'}): 38,
  frozenset({'B', 'D'}): 37,
  frozenset({'A', 'D'}): 44,
  frozenset({'D', 'E'}): 24,
  frozenset({'B', 'C'}): 44,
  frozenset({'A', 'C'}): 44,
  frozenset({'C', 'E'}): 38,
  frozenset({'A', 'B'}): 50,
  frozenset({'B', 'E'}): 41,
  frozenset({'A', 'E'}): 44,
  frozenset({'B', 'C', 'D'}): 79.33333333333333,
  frozenset({'A', 'C', 'D'}): 84.0,
  frozenset({'C', '

In [10]:
# 必要なライブラリのインポート
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# データの読み込み
file_path = "GPTニキ用３.xlsx"  # 最新のアップロードファイルを使用
excel_data = pd.ExcelFile(file_path)
df = excel_data.parse(excel_data.sheet_names[0])

# 説明変数と目的変数の定義
X = df[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]
y = df['目的値']

# 線形回帰モデルの作成とトレーニング
model = LinearRegression()
model.fit(X, y)

# 線形回帰モデルの予測値を計算
y_pred = model.predict(X)



# 回帰係数の取得
coefficients = model.coef_
intercept = model.intercept_

# 辞書形式で保存（カラム名をキーにする）
coef_dict = {
    'intercept': intercept
}
for feature, coef in zip(X.columns, coefficients):
    coef_dict[feature] = coef

# 辞書の表示
print(coef_dict)







{'intercept': -311.7333333333336, 'A': 11.57793148880104, 'B': 3.616798418972161, 'C': -2.0470685111989275, 'D': -3.7970685111988898, 'E': -0.1720685111988829, 'F': 5.561725955204345, 'G': 3.616798418972178, 'H': 9.1785243741764, 'I': 295.3212121212124}


In [1]:
import pulp

# 工場と月のリスト
factories = ['工場A', '工場B']
months = ['4月', '5月']

tuple_list = [(factory,month) for factory in factories for month in months]

# pulp の LpVariable.dicts を用いて変数を作成（各変数は LpVariable オブジェクトになる）
data = pulp.LpVariable.dicts("FactoryData", tuple_list, lowBound=0, cat='Continuous')

# モデルの解を得た後、値を取得したい場合
nested_dict = {}
data


{('工場A', '4月'): FactoryData_('工場A',_'4月'),
 ('工場A', '5月'): FactoryData_('工場A',_'5月'),
 ('工場B', '4月'): FactoryData_('工場B',_'4月'),
 ('工場B', '5月'): FactoryData_('工場B',_'5月')}

In [2]:
# 各キーとその値を取得
for key in data.keys():
    factory, month = key  # キーをアンパック
    value = data[factory, month].value()  # 各変数の値を取得

    # 入れ子の辞書を作成
    if factory not in nested_dict:
        nested_dict[factory] = {}
    
    nested_dict[factory][month] = value

print(nested_dict)

{'工場A': {'4月': None, '5月': None}, '工場B': {'4月': None, '5月': None}}


In [1]:
import pandas as pd

df = pd.read_excel("./ver370用最終_4実8推プレ_工場月度割り当て_パラメータ田村さん記載.xlsx",sheet_name="生産能力")

In [2]:
df

,工程時間内訳,工場,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月
0,暦日日数,L1,0,0,0,0,30,31,31.0,29,28,35,28.0,31
1,暦日時間,L1,0,0,0,0,720,744,744.0,696,672,840,672.0,744
2,休日時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
3,非操業時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
4,救済・再梱包時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,保全/工事,L7,0,0,0,0,144,0,0.0,0,480,0,0.0,160
66,開発テスト,L7,0,0,0,0,0,0,0.0,0,0,0,200.0,0
67,生技テスト,L7,0,0,0,0,8,24,12.0,0,24,12,0.0,24
68,計画停止,L7,0,0,0,0,0,0,0.0,0,0,0,0.0,0


In [3]:
df[df["工場"] == "L1"]

,工程時間内訳,工場,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月
0,暦日日数,L1,0,0,0,0,30,31,31.0,29,28,35,28.0,31
1,暦日時間,L1,0,0,0,0,720,744,744.0,696,672,840,672.0,744
2,休日時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
3,非操業時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
4,救済・再梱包時間,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
5,保全/工事,L1,0,0,0,0,0,0,0.0,264,216,0,240.0,0
6,開発テスト,L1,0,0,0,0,720,468,48.0,0,0,0,0.0,0
7,生技テスト,L1,0,0,0,0,0,36,0.0,0,24,0,0.0,0
8,計画停止,L1,0,0,0,0,0,0,0.0,0,0,0,0.0,0
9,負荷時間,L1,0,0,0,0,0,240,696.0,432,432,840,432.0,744


In [4]:
nested_dict = {}
for _, row in df.iterrows():
    factory = row['工場']
    metric = row['工程時間内訳']
    if factory not in nested_dict:
        nested_dict[factory] = {}
    for month in df.columns[2:]:
        if month not in nested_dict[factory]:
            nested_dict[factory][month] = {}
        nested_dict[factory][month][metric] = row[month]

nested_dict

{'L1': {'4月': {'暦日日数': 0,
   '暦日時間': 0,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 0,
   '生技テスト': 0,
   '計画停止': 0,
   '負荷時間': 0},
  '5月': {'暦日日数': 0,
   '暦日時間': 0,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 0,
   '生技テスト': 0,
   '計画停止': 0,
   '負荷時間': 0},
  '6月': {'暦日日数': 0,
   '暦日時間': 0,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 0,
   '生技テスト': 0,
   '計画停止': 0,
   '負荷時間': 0},
  '7月': {'暦日日数': 0,
   '暦日時間': 0,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 0,
   '生技テスト': 0,
   '計画停止': 0,
   '負荷時間': 0},
  '8月': {'暦日日数': 30,
   '暦日時間': 720,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 720,
   '生技テスト': 0,
   '計画停止': 0,
   '負荷時間': 0},
  '9月': {'暦日日数': 31,
   '暦日時間': 744,
   '休日時間': 0,
   '非操業時間': 0,
   '救済・再梱包時間': 0,
   '保全/工事': 0,
   '開発テスト': 468,
   '生技テスト': 36,
   '計画停止': 0,
   '負荷時間': 240},
  '10月': {'暦日日数': 31.0,
   '暦日時間': 744.0,
   '休日時間'

In [5]:
nested_dict["L1"]

{'4月': {'暦日日数': 0,
  '暦日時間': 0,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 0,
  '生技テスト': 0,
  '計画停止': 0,
  '負荷時間': 0},
 '5月': {'暦日日数': 0,
  '暦日時間': 0,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 0,
  '生技テスト': 0,
  '計画停止': 0,
  '負荷時間': 0},
 '6月': {'暦日日数': 0,
  '暦日時間': 0,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 0,
  '生技テスト': 0,
  '計画停止': 0,
  '負荷時間': 0},
 '7月': {'暦日日数': 0,
  '暦日時間': 0,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 0,
  '生技テスト': 0,
  '計画停止': 0,
  '負荷時間': 0},
 '8月': {'暦日日数': 30,
  '暦日時間': 720,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 720,
  '生技テスト': 0,
  '計画停止': 0,
  '負荷時間': 0},
 '9月': {'暦日日数': 31,
  '暦日時間': 744,
  '休日時間': 0,
  '非操業時間': 0,
  '救済・再梱包時間': 0,
  '保全/工事': 0,
  '開発テスト': 468,
  '生技テスト': 36,
  '計画停止': 0,
  '負荷時間': 240},
 '10月': {'暦日日数': 31.0,
  '暦日時間': 744.0,
  '休日時間': 0.0,
  '非操業時間': 0.0,
  '救済・再梱包時間': 0.0,
  '保全/工事': 0.0,
  '開発テスト': 